# Two-Stage Object detection w/ Freeze Learning & Fine-tuning
For this approach, it will be used a pretrained DensetNet201 as a backbone for a F-RCNN model for object detection, to which it will be applied freeze learning and fine-tuning to our task.

In [1]:
import os
from PIL import Image
import torch 
import pandas as pd

DATA_DIR = "../data/"
IMG_DIR = DATA_DIR + "/images/"
ANNOTATION_DIR = DATA_DIR + "/annotations/"
SPLITS_DIR = DATA_DIR + "/dl-split/"
OUT_DIR = "./out/yolo_obj_detect/"

os.makedirs(OUT_DIR, exist_ok=True)

SEED = 42

# Model Definition

In [2]:
!git clone https://github.com/ultralytics/yolov5 models/yolov5

fatal: destination path 'models/yolov5' already exists and is not an empty directory.


In [3]:
import numpy as np
from torch.utils.data import SubsetRandomSampler

np.random.seed(SEED)

train_names = []
with open('../data/dl-split/train.txt', 'r') as train_f:
    train_names = train_f.readlines()
    train_names = [x.strip() for x in train_names]

train_indices = list(range(len(train_names)))
np.random.shuffle(train_indices)
train_val_split = int(np.floor(0.2 * len(train_indices)))

train_idx, val_idx = train_indices[train_val_split:], train_indices[:train_val_split]

val_data = [train_names[idx] for idx in val_idx]
train_data = [train_names[idx] for idx in train_idx]

test_data = []
with open('../data/dl-split/test.txt', 'r') as test_f:
    test_data = test_f.readlines()
    test_data = [x.strip() for x in train_names]



# Prepare Folders

In [4]:
from utils.utils import parse_annotation
import shutil

TRAIN_DIR='../data/images/train'
TEST_DIR='../data/images/test'
VAL_DIR='../data/images/val'

TRAIN_LABELS='../data/labels/train'
TEST_LABELS='../data/labels/test'
VAL_LABELS='../data/labels/val'

os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)
os.makedirs(VAL_DIR, exist_ok=True)

os.makedirs(TRAIN_LABELS, exist_ok=True)
os.makedirs(TEST_LABELS, exist_ok=True)
os.makedirs(VAL_LABELS, exist_ok=True)

label_encode_map = {
    "trafficlight": 0,
    "speedlimit": 1,
    "crosswalk": 2,
    "stop": 3,
}

def transform_labels(out_path: str, annot_dict):
    annot_file = open(out_path, 'w')
    
    img_width = annot_dict['width']
    img_height = annot_dict['height']
    for label, box in zip(annot_dict['labels'], annot_dict['boxes']):
        width = (box[2]-box[0])/img_width
        height = (box[3]-box[1])/img_height
        
        x_center = (box[0]+box[2])/2/img_width
        y_center = (box[3]+box[1])/2/img_height
        
        annot_file.write(f'{label} {x_center} {y_center} {width} {height}\n')
    annot_file.close()

# training data
for name in train_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{TRAIN_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=TRAIN_DIR)
  
# validation data      
for name in val_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{VAL_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=VAL_DIR)
 
# test data       
for name in test_data:
    annot_dict = parse_annotation(f'../data/annotations/{name}.xml', label_encode_map)
    transform_labels(f'{TEST_LABELS}/{name}.txt', annot_dict)
    shutil.copy(src=f'../data/images/{name}.png', dst=TEST_DIR)

# Train Model

In [19]:
!python models/yolov5/train.py -h

usage: train.py [-h] [--weights WEIGHTS] [--cfg CFG] [--data DATA] [--hyp HYP]
                [--epochs EPOCHS] [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
                [--rect] [--resume [RESUME]] [--nosave] [--noval]
                [--noautoanchor] [--noplots] [--evolve [EVOLVE]]
                [--bucket BUCKET] [--cache [CACHE]] [--image-weights]
                [--device DEVICE] [--multi-scale] [--single-cls]
                [--optimizer {SGD,Adam,AdamW}] [--sync-bn] [--workers WORKERS]
                [--project PROJECT] [--name NAME] [--exist-ok] [--quad]
                [--cos-lr] [--label-smoothing LABEL_SMOOTHING]
                [--patience PATIENCE] [--freeze FREEZE [FREEZE ...]]
                [--save-period SAVE_PERIOD] [--local_rank LOCAL_RANK]
                [--entity ENTITY] [--upload_dataset [UPLOAD_DATASET]]
                [--bbox_interval BBOX_INTERVAL]
                [--artifact_alias ARTIFACT_ALIAS]

options:
  -h, --help            show this help message a

In [6]:
!python models/yolov5/train.py --batch 4 --epochs 20 --data models/yolo_cfg.yaml --workers 2 --project out/yolo/ --optimizer SGD --hyp models/hyp.yaml

# Test Model

In [7]:
!python models/yolov5/detect.py -h 

usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE]
                 [--data DATA] [--imgsz IMGSZ [IMGSZ ...]]
                 [--conf-thres CONF_THRES] [--iou-thres IOU_THRES]
                 [--max-det MAX_DET] [--device DEVICE] [--view-img]
                 [--save-txt] [--save-conf] [--save-crop] [--nosave]
                 [--classes CLASSES [CLASSES ...]] [--agnostic-nms]
                 [--augment] [--visualize] [--update] [--project PROJECT]
                 [--name NAME] [--exist-ok] [--line-thickness LINE_THICKNESS]
                 [--hide-labels] [--hide-conf] [--half] [--dnn]

options:
  -h, --help            show this help message and exit
  --weights WEIGHTS [WEIGHTS ...]
                        model path(s)
  --source SOURCE       file/dir/URL/glob, 0 for webcam
  --data DATA           (optional) dataset.yaml path
  --imgsz IMGSZ [IMGSZ ...], --img IMGSZ [IMGSZ ...], --img-size IMGSZ [IMGSZ ...]
                        inference size h,w
  --co

In [4]:
!python models/yolov5/detect.py --weights out/yolo/exp4/weights/best.pt --source ../data/images/test/*.png --project out/yolo/runs --save-txt --conf-thres 0.4 --save-conf

detect: weights=['out/yolo/exp4/weights/best.pt'], source=../data/images/test/*.png, data=models\yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=out/yolo/runs, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
fatal: cannot change to 'C:\Users\Ivo': No such file or directory
YOLOv5  2022-6-9 Python-3.10.1 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 950, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients
image 1/613 C:\Users\Ivo Saavedra\Projects\traffic-signifier\data\images\test\road0.png: 640x448 1 trafficlight, Done. (0.021s)
image 2/613 C:\Users\Ivo Saavedra\Projects\traffic-signifier\data\images\test\road1.png: 480x640 1 trafficlight, Done. (0.021s)
image 3/613 C:\Users\Ivo Saave

In [5]:
!python models/yolov5/val.py -h 

usage: val.py [-h] [--data DATA] [--weights WEIGHTS [WEIGHTS ...]]
              [--batch-size BATCH_SIZE] [--imgsz IMGSZ]
              [--conf-thres CONF_THRES] [--iou-thres IOU_THRES] [--task TASK]
              [--device DEVICE] [--workers WORKERS] [--single-cls] [--augment]
              [--verbose] [--save-txt] [--save-hybrid] [--save-conf]
              [--save-json] [--project PROJECT] [--name NAME] [--exist-ok]
              [--half] [--dnn]

options:
  -h, --help            show this help message and exit
  --data DATA           dataset.yaml path
  --weights WEIGHTS [WEIGHTS ...]
                        model.pt path(s)
  --batch-size BATCH_SIZE
                        batch size
  --imgsz IMGSZ, --img IMGSZ, --img-size IMGSZ
                        inference size (pixels)
  --conf-thres CONF_THRES
                        confidence threshold
  --iou-thres IOU_THRES
                        NMS IoU threshold
  --task TASK           train, val, test, speed or study
  --device D

In [12]:
!python models/yolov5/val.py --workers 2 --verbose --task test --weights out/yolo/exp4/weights/best.pt --data models/yolo_cfg.yaml --batch-size 4 --conf-thres=0.25 --iou-thres 0.4  --save-json

loading annotations into memory...


val: data=models/yolo_cfg.yaml, weights=['out/yolo/exp4/weights/best.pt'], batch_size=4, imgsz=640, conf_thres=0.25, iou_thres=0.4, task=test, device=, workers=2, single_cls=False, augment=False, verbose=True, save_txt=False, save_hybrid=False, save_conf=False, save_json=True, project=models\yolov5\runs\val, name=exp, exist_ok=False, half=False, dnn=False
fatal: cannot change to 'C:\Users\Ivo': No such file or directory
YOLOv5  2022-6-9 Python-3.10.1 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce GTX 950, 2048MiB)

Fusing layers... 
Model summary: 213 layers, 7020913 parameters, 0 gradients

test: Scanning 'C:\Users\Ivo Saavedra\Projects\traffic-signifier\data\labels\test.cache' images and labels... 613 found, 0 missing, 0 empty, 0 corrupt: 100%|██████████| 613/613 [00:00<?, ?it/s]
test: Scanning 'C:\Users\Ivo Saavedra\Projects\traffic-signifier\data\labels\test.cache' images and labels... 613 found, 0 missing, 0 empty, 0 corrupt: 100%|██████████| 613/613 [00:00<?, ?it/s]

               Cl